In [15]:
using AdvancedHMC, ForwardDiff, LinearAlgebra


make_sym(A) = (transpose(A) + A) / 2.

n_samples = 1000
n_adapts = 100
D = 28
kernel = Matrix{Float64}(I, D, D)
data = zeros(D)
data[10] = 1
data[20] = -5
data_errors = ones(D) / 10
data_errors = cat(data_errors...; dims=(1,2))
sig_inv = Matrix{Float64}(I, D, D) * 10

initial_θ = zeros(D)

ℓπ = let D=D, sig_inv=sig_inv, data=data, data_errors=data_errors, kernel=kernel
        phi -> begin
            mu = zeros(D)
            pr_log = -1/2 * transpose(phi - mu) * sig_inv * (phi - mu)
            mu_ = kernel * phi
            li_log = -1/2 * transpose(data - mu_) * make_sym(pinv(data_errors)) * (data - mu_)
        return pr_log + li_log
        end
    end


#23 (generic function with 1 method)

In [16]:
# ℓπ = let D=D, kernel=kernel, data=data, data_errors=data_errors
#         phi -> begin
#             mu = zeros(D)
#             prior_log = -1/2 * transpose(phi - mu) * sig_inv * (phi - mu)
#             mu_ = kernel * phi
#             likelihood_log = -1/2 * transpose(data - mu_) * make_sym(pinv(data_errors)) * (data - mu_)
#             return prior_log + likelihood_log
#         end
#     end

# metric = DiagEuclideanMetric(D)
# hamiltonian = Hamiltonian(metric, posterior, ForwardDiff)
# initial_ϵ = find_good_eps(hamiltonian, initial_θ)
# integrator = Leapfrog(initial_ϵ)
# proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
# adaptor = StanHMCAdaptor(Preconditioner(metric), NesterovDualAveraging(0.8, integrator))
# samples, stats = AdvancedHMC.sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=false);

metric = DiagEuclideanMetric(D)
hamiltonian = Hamiltonian(metric, ℓπ, ForwardDiff)
initial_ϵ = find_good_eps(hamiltonian, initial_θ) 
integrator = Leapfrog(initial_ϵ)
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(Preconditioner(metric), NesterovDualAveraging(0.8, integrator))
samples, stats = AdvancedHMC.sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=false);

┌ Info: Finished 100 adapation steps
│   adaptor = StanHMCAdaptor(
    pc=DiagPreconditioner,
    ssa=NesterovDualAveraging(γ=0.05, t_0=10.0, κ=0.75, δ=0.8, state.ϵ=0.14113716732267156),
    init_buffer=75, term_buffer=50, window_size=25,
    state=window(76, 50), window_splits()
)
│   τ.integrator = Leapfrog(ϵ=0.141)
│   h.metric = DiagEuclideanMetric([1.0, 1.0, 1.0, 1.0, 1.0, 1 ...])
└ @ AdvancedHMC /Users/ta_nyan/.julia/packages/AdvancedHMC/Sn9Ek/src/sampler.jl:159
┌ Info: Finished 1000 sampling steps for 1 chains in 5.811323382 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([1.0, 1.0, 1.0, 1.0, 1.0, 1 ...]))
│   τ = NUTS{MultinomialTS,Generalised}(integrator=Leapfrog(ϵ=0.141), max_depth=10), Δ_max=1000.0)
│   EBFMI_est = 0.8829370679859613
│   average_acceptance_rate = 0.8460698592096328
└ @ AdvancedHMC /Users/ta_nyan/.julia/packages/AdvancedHMC/Sn9Ek/src/sampler.jl:178


In [17]:
using Plots
plotly()



Plots.PlotlyBackend()

In [18]:
coeff = [[samples[j][i] for j in range(1, stop=n_samples)] for i in eachindex(samples[1])];

In [19]:
histogram(coeff[5])

<!DOCTYPE html>
 
 
 Plots.jl

In [20]:
using Distributions
mean(coeff[5])

0.006637851149802179

In [21]:
coeff_plot = [mean(coeff[i]) for i in range(1, stop=28)]
err_plot = [cov(coeff[i]) for i in range(1, stop=28)]
plot(coeff_plot, ribbon=err_plot, fillalpha=0.3)

<!DOCTYPE html>
 
 
 Plots.jl